In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import torch
from torch import nn

from data_generation import generate_toy_dataset, pad_sequences, \
    one_hot_encode_sequences
from utils import batch_generator, display_metrics, compare_sequences, \
    visualize_attention, calculate_loss, calculate_metric, train
from models import EncoderDecoder, EncDecAttnDotProduct, EncDecAttnBilinear, EncDecAttnConcat

%matplotlib inline

In [ ]:
torch.__version__

Зададим параметр инициализации генератора случайных чисел для возможности воспроизведения результатов экспериментов.

In [ ]:
torch.manual_seed(0)
np.random.seed(1)

# Генерация данных
Для демонстрации работы Механизма Внимания воспользуемся синтетическими данными. В качестве входной последовательности будет выступать последовательность из букв латинского алфавита с повторениями, а на выходе - та же последовательность в исходном порядке.

Используемые данные нужны исключительно для демонстрации работы моделей. В реальном приложении данные могли бы представлять:
* последовательность пар предложений, где второе предложение является переводом первого на другой естественный язык (задача машинного перевода);
* последовательность пар "запрос-ответ" (чат-боты);
* большой текст и "выжимка" из него (краткое изложение содержания) и др.

Сгенерируем датасет со следующими параметрами:
* Словарь - буквы латинского алфавита. Будем считать, что токен в нашей последовательности - один символ.
* Минимальная длина последовательности - 2, максимальная - 10.
* Число записей - 15000.

In [ ]:
vocabulary = np.array(list('abcdefghijklmnopqrstuvwxyz'))
print("Vocabulary:", vocabulary)

In [ ]:
SEQ_MAX_LEN = 10
copy_dataset = generate_toy_dataset(vocabulary, seq_min_len=2,
                                    seq_max_len=SEQ_MAX_LEN,
                                    seq_count=15000)

Посмотрим на данные:

In [ ]:
copy_dataset.head()

Теперь сформируем последовательности одинаковой длины `sequence_max_size` + 2:
* В начале каждой последовательности поставим символ `bos` (begin of sequence).
* В конце последовательности поставим символ `eos` (end of sequence). Если итоговая длина полученной последовательности получилась меньше требуемой, добавляем `eos` до тех пор, пока не получим требуемую длину.

Возникает закономерный вопрос, зачем мы это делаем.
1. Декодеру нужно сообщить о начале последовательности. Свое первое состояние декодер берет из последнего состояния энкодера, а что взять в качестве первого входного токена для декодера? Символ начала последовательности `bos`.
2. Работа декодера должна завершатся, когда сгенерирована вся последовательность. Для этого мы будем учить его предсказывать специальный символ конца последовательности `eos`. 
3. Процедура формирования последовательностей одинаковой длины необходима, так как мы хотим обучать сеть батчами, а не по одному примеру прогонять за раз.

Для обучения сети батчами нам также понадобятся бинарные маски для последовательностей. Вектор маски имеет тот же размер, что и последовательность, при этом нули в ней сооветствуют лишним символам `eos`, а единицы всем остальным символам. Данные маски понадобятся при определении лосс-функции и для реализации экодера. Бинарные маски для входной последовательности содержатся в поле `mask_inference_input`.

In [ ]:
BOS = "*"
EOS = "#"
copy_dataset = pad_sequences(copy_dataset, SEQ_MAX_LEN, BOS, EOS)

In [ ]:
copy_dataset.head()

Работать с символами напрямую мы не можем, переводим символы в числа. Закодируем символы следующим образом:
* bos закодируем числом 0;
* eos закодируем числом 1;
* все остальные символы закодируем по порядку.

In [ ]:
BOS_CODE = 0
EOS_CODE = 1
sym2num = {sym: i+2 for i, sym in enumerate(vocabulary)}
sym2num.update({BOS: BOS_CODE, EOS: EOS_CODE})

Теперь вместо последовательности токенов фиксированной длины имеем последовательность фиксированной длины из чисел. Произведем One Hot кодирование этой числовой последовательности.

Всего в закодированной исходной числовой последовательности `N=28` различных чисел (2 для специальных символов и 26 для символов алфавита). Для кажого числа `x` в исходной последовательности поставим в соответсвтвие вектор размерности 28 из нулей и одной единицы на позиции = числу `x`.

Пусть, например, мы кодируем последовательность `[*, a, #, #]` или `[0, 2, 1, 1]`, причем `N=4`. Тогда число `x=0` будем кодировать вектором `[1, 0, 0, 0]`, `x=1` вектором `[0, 1, 0, 0]`, `x=2` вектором `[0, 0, 1, 0]`.
Полученные векторы сложим в одну матрицу и получим 
`[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0]]` - закодированную последовательность.

In [ ]:
copy_dataset = one_hot_encode_sequences(copy_dataset, sym2num)
copy_dataset.head()

В данном случае мы не воспользовались методами класса <a href=https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html>OneHotEncoder</a> из `sklearn.preprocessing` с целью пошаговой подробной демонстрации происходящего. Разумеется, когда вы будете решать не учебные задачи, вам нужно будет пользоваться уже готовыми реализациями. Также вряд ли вам когда-либо надо будет иметь дело с one-hot векторами напрямую, а не с их sparse-версиями (возможность создания sparse векторов также есть у `OneHotEncoder`).

Произведем разбиение датасета на обучающую выборку, валидацию и тест.

In [ ]:
train_data, test_data = train_test_split(copy_dataset, test_size=0.25,
                                         random_state=10)
val_data, test_data = train_test_split(test_data, test_size=0.25,
                                       random_state=11)

Отметим также, что для обучения рекуррентых сетей батчами в pytorch есть специальные функции:
1. <a href=https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html>pad_sequence</a>: позволяет получать последовательности одинаковой длины в батче.
2. <a href=https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pack_padded_sequence>pack_padded_sequence</a>: позволяет не совершать лишних итераций рекуррентной сети.

<a href=https://stackoverflow.com/a/55805785/12919840>Тут</a> можно почитать про это.

# Модель Encoder-Decoder

Базовый класс `BaseEncoderDecoder` определен в файле `model.py`.
1. Кодирование (`self.encode`) исходной последовательности осуществляется при помощи линейного слоя `self.embedding`. Обратите внимание на параметры этого линейного слоя (какая входная размерность векторов, какая выходная).
   * Матрица весов слоя `self.embedding` умножается на закодированную one-hot последовательность, что позволяет перейти от бинарных векторов для токенов к действительным векторам.
   * Эта операция может быть весьма затратной в той реализации, которая представлена здесь. Дело в том, что словарь для реальных данных обычно состоит не из 26 токенов (+2 токена bos и eos), а из десятков тысяч токенов. В этом случае каждый one-hot вектор для токена будет содержать десятки тысяч значений, и матрица весов слоя `self.embedding` будет перемножаться с вектором большой размерности, что весьма вычислительно затратно.
   * С другой стороны, зачем нам вообще нужны one-hot вектора и матричное умножение? Ведь умножение one-hot вектора на матрицу равносильно просто выбору строки из данной матрицы. Слой <a href=https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html>Embedding</a> из `torch.nn` именно это и делает. При решении не учебной задачи можно воспользоваться <a href=https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html#sklearn.preprocessing.OrdinalEncoder>OrdinalEncoder</a> из `sklearn.preprocessing` для кодирования токенов в натуральные числа, а в самой сети добавить слой `Embedding` (см. документацию).
2. В коде, где происходит генерация скрытых состояний кодируемой последовательности присутствует такая строчка:
```
# save new state for not eos tokens, otherwise save prev state
state = torch.where(
    torch.tile(mask_inference_inputs[:, i, None],
               [1, next_state.shape[1]]),
    next_state, state
)
```
Зачем нужен этот код? Впишите свой ответ в ячейке ниже.

<b>Ответ:</b> 

3. Как видно из кода модели, декодирование в режиме обучения (когда `self.training` == True) и режиме инференса (в противном случае) осуществляется при помощи разных методов (`self.decode_training` и `self.decode_eval`). Почему? Впишите свой ответ в ячейке ниже.

<b>Ответ:</b>

4. Какая используется стратегия декодирования на этапе инференса? Какие еще стратегии декодирования вы знаете?

<b>Ответ:</b>

5. На вход энкодеру приходит вектор размерности `embed_size + len(mapping)`. Почему? Какие дополнительные признаки были добавлены к эмбеддинговым признакам в коде?

<b>Ответ:</b>

## Без Механизма Внимания

Определим первую модель, с которой проведем эксперимент. Это обычная модель Encoder-Decoder без использования Механизма Внимания.

In [ ]:
enc_dec_model = EncoderDecoder(sym2num, BOS, EOS, embed_size=50,
                               enc_hidden_size=70, dec_hidden_size=70)
optimizer = torch.optim.Adam(enc_dec_model.parameters())

Для оценки качества модели воспользуемся расстоянием Левенштейна, деленным на максимальную длину из длин истинной или предсказанной последовательности. Чем меньше расстояние между этими последовательностями, тем модель делает более точные предсказания.

Всю информацию о процессе обучения смотри в файле `utils.py`, функция `train`.

In [ ]:
try:
    train(enc_dec_model, train_data, val_data, optimizer, SEQ_MAX_LEN,
          batch_size=50, epochs_count=80)
except KeyboardInterrupt:
    pass

Посмотрим на предсказания:

In [ ]:
compare_sequences(enc_dec_model, val_data, SEQ_MAX_LEN)

<b>Домашняя работа:</b>
1. Проведите несколько экспериментов с данной моделью. Саму модель не надо менять, меняйте параметры оптимизатора, число эпох обучения и размер батча.
2. Получите качество лучшей модели на тестовой выборке.
3. Сделайте выводы, в которых в том числе отметьте: сколько эпох потребовалось для обучения, какое финальное качество модели вы получили.

Теперь давайте добавим Механизм Внимания.

## С добавлением Механизма Внимания

<b>Домашняя работа</b>: реализовать требуемые классы в файле `models.py` и обучить модели.

Перед реализацией моделей посмотри код для визуализации весов Внимания: файл `utils.py`, функция  `visualize_attention`.
В коде моделей есть `attn_weights` поле, которое вы должны заполнить.

### Скалярное произведение (Dot-Product)
Реализуйте Encoder-Decoder сеть с Механизмом Внимания в виде простого скалярного произведения (класс `EncDecAttnDotProduct`).
$$logit_i = <enc_{i}, dec>, $$
где $i \in \overline{0, N_{e}}$, $N_{e}$ - число состояний энкодера.

<b>! Обратите внимание</b>, что полученные логиты еще не являются весами Внимания. Для получения весов нужно:
1. Наложить бинарную маску: те логиты, которые относятся к дополнительным `eos` токенам приравняйте к $-\infty$, т.е. к $-10^{9}$.
2. Применить Softmax.

In [ ]:
enc_dec_dot_product = EncDecAttnDotProduct(
    sym2num, BOS, EOS, embed_size=50, enc_hidden_size=70,
    dec_hidden_size=70)
optimizer = torch.optim.Adam(enc_dec_dot_product.parameters(), lr=0.0005)

In [ ]:
try:
    train(enc_dec_dot_product, train_data, val_data, optimizer, SEQ_MAX_LEN,
          batch_size=50, epochs_count=5)
except KeyboardInterrupt:
    pass

In [ ]:
compare_sequences(enc_dec_dot_product, val_data, SEQ_MAX_LEN)

In [ ]:
visualize_attention(enc_dec_dot_product, val_data, SEQ_MAX_LEN, BOS, EOS,
                    batch_size=2)

<b>Домашняя работа:</b>
1. Проведите несколько экспериментов с данной моделью. Саму модель не надо менять, меняйте параметры оптимизатора, число эпох обучения и размер батча.
2. Получите качество лучшей модели на тестовой выборке.
3. Визуализируйте веса Внимания лучшей модели.
4. Сделайте выводы, в которых в том числе отметьте: сколько эпох потребовалось для обучения, какое финальное качество модели вы получили. Лучше ли данная модель с точки зрения определенной метрики на тестовой выборки, чем Encoder-Decoder без внимания? Отличается ли число параметров текущей модели от числа параметров модели без внимания? О чем это говорит?

### Билинейное Внимание

Реализуйте Encoder-Decoder сеть с Механизмом Внимания в следующем виде (класс `EncDecAttnBilinear`):
1. К состоянию энкодера применяется линейное преобразование.
2. Затем считается скалярное произведение между обновленным состоянием энкодера и состоянием декодера.

$$logit_i = <linear(enc_{i}), dec>, $$
где $i \in \overline{0, N_{e}}$, $N_{e}$ - число состояний энкодера.

<b>! Обратите внимание</b>, что полученные логиты еще не являются весами Внимания. Для получения весов нужно:
1. Наложить бинарную маску: те логиты, которые относятся к дополнительным `eos` токенам приравняйте к $-\infty$, т.е. к $-10^{9}$.
2. Применить Softmax.

In [ ]:
enc_dec_bilinear = EncDecAttnBilinear(
    sym2num, BOS, EOS, embed_size=50, enc_hidden_size=70,
    dec_hidden_size=70)
optimizer = torch.optim.Adam(enc_dec_bilinear.parameters(), lr=0.0005)

In [ ]:
try:
    train(enc_dec_bilinear, train_data, val_data, optimizer, SEQ_MAX_LEN,
          batch_size=50, epochs_count=11)
except KeyboardInterrupt:
    pass

In [ ]:
compare_sequences(enc_dec_bilinear, val_data, SEQ_MAX_LEN)

In [ ]:
visualize_attention(enc_dec_bilinear, val_data, SEQ_MAX_LEN, BOS, EOS,
                    batch_size=2)

<b>Домашняя работа:</b>
1. Проведите несколько экспериментов с данной моделью. Саму модель не надо менять, меняйте параметры оптимизатора, число эпох обучения и размер батча.
2. Получите качество лучшей модели на тестовой выборке.
3. Визуализируйте веса Внимания лучшей модели.
4. Сделайте выводы, в которых в том числе отметьте: сколько эпох потребовалось для обучения, какое финальное качество модели вы получили.

### Конкатенакция

Реализуйте Encoder-Decoder сеть с Механизмом Внимания в следующем виде (класс `EncDecAttnConcat`):
1. Состояние энкодера конкатенируется с состоянием декодера.
2. Применяется линейный слой (выходная размерность вектора = `dec_hidden_size`).
3. Применяется тангенс гиперболический.
4. Осуществляется скалярное произведение с обучаемым вектором.

$$concat_i = concatenate(enc_{i}, dec)$$
$$logit_i = <tanh(linear(concat_i)), vec>$$
где $i \in \overline{0, N_{e}}$, $N_{e}$ - число состояний энкодера.

<b>! Обратите внимание</b>, что полученные логиты еще не являются весами Внимания. Для получения весов нужно:
1. Наложить бинарную маску: те логиты, которые относятся к дополнительным `eos` токенам приравняйте к $-\infty$, т.е. к $-10^{9}$.
2. Применить Softmax.

In [ ]:
enc_dec_concat = EncDecAttnConcat(
    sym2num, BOS, EOS, embed_size=50, enc_hidden_size=70,
    dec_hidden_size=70)
optimizer = torch.optim.Adam(enc_dec_concat.parameters(), lr=0.0005)

In [ ]:
try:
    train(enc_dec_concat, train_data, val_data, optimizer, SEQ_MAX_LEN,
          batch_size=50, epochs_count=15)
except KeyboardInterrupt:
    pass

In [ ]:
compare_sequences(enc_dec_concat, val_data, SEQ_MAX_LEN)

In [ ]:
visualize_attention(enc_dec_concat, val_data, SEQ_MAX_LEN, BOS, EOS, batch_size=2)

<b>Домашняя работа:</b>
1. Проведите несколько экспериментов с данной моделью. Саму модель не надо менять, меняйте параметры оптимизатора, число эпох обучения и размер батча.
2. Получите качество лучшей модели на тестовой выборке.
3. Визуализируйте веса Внимания лучшей модели.
4. Сделайте выводы, в которых в том числе отметьте: сколько эпох потребовалось для обучения, какое финальное качество модели вы получили.